In [14]:
import csv
from scapy.all import sniff, IP, TCP, UDP, ICMP, Raw
def write_to_csv(data):
    with open('network_packets.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(data)
def packet_callback(packet):
    # Initialize common variables
    source_ip = target_ip = source_port = target_port = packet_size = request_mode = packet_length = 'N/A'
    packet_summary = ""

    # Check if the packet has the IP layer
    if IP in packet:
        source_ip = packet[IP].src
        target_ip = packet[IP].dst

    # Check for TCP, UDP, and ICMP
    if TCP in packet:
        source_port = packet[TCP].sport
        target_port = packet[TCP].dport
        packet_size = len(packet[TCP])
        request_mode = 'TCP'
        packet_summary = packet.summary()
        # Check for HTTP by looking for packets typically on port 80 or 443 and carrying a payload
        if packet[TCP].dport in [80, 443] and Raw in packet:
            payload = packet[Raw].load.decode(errors='ignore')
            if 'HTTP' in payload:
                request_mode = 'http'
                packet_summary += ' | HTTP Payload: ' + payload[:40]  # Show first 40 characters of payload
    elif UDP in packet:
        source_port = packet[UDP].sport
        target_port = packet[UDP].dport
        packet_size = len(packet[UDP])
        request_mode = 'udp'
        packet_summary = packet.summary()
    elif ICMP in packet:
        packet_size = len(packet[ICMP])
        request_mode = 'icmp'
        packet_summary = packet.summary()
    else:
        packet_summary = packet.summary()
    # Packet length might be considered as the entire packet length
    packet_length = len(packet)
    # Append the data to the CSV
    data = [
        packet.time, source_ip, target_ip, source_port, target_port,
        packet_size, request_mode, packet_length, packet_summary
    ]
    write_to_csv(data)
with open('network_packets.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([
        'Timestamp', 'Source IP', 'Target IP', 'Source Port', 'Target Port',
        'Packet Size', 'Request Mode', 'Packet Length', 'Packet Summary'
    ])
# Start sniffing packets
sniff(prn=packet_callback, store=False)

<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>